In [171]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [177]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


documents=pd.read_csv('jd_source_c_2.csv')

documents['documents'] = documents['job_desc'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
documents_df=pd.DataFrame(documents,columns=['documents'])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\icyro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [178]:
documents_df

,documents
0,"Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support"
1,"Software Engineer/ Data Engineer – Permanent Position Requirements: Degree Information Technology, Computer Science, Biotechnology, Science equivalent. At least 5 years research experience field computer science, Data Preparation, Mathematical modelling Simulation, Data Analytics Optimization. Knowledge machine learning data mining techniques one areas statistical modeling methods, time series, natural language, image/video text mining, optimization, information retrieval. Proficient Python, Excel, Scala, R, Matlab. Proficient Data science algorithms strong understanding field. Have experience web development (HTML, CSS, JS). Familarity Linux. Able work independently develop & operate app Hadoop environment. Familiar Spark, HBase, Hive, Hadoop Big Data related technologies. Experience RHEL Linux, RDBMS SQL essential. Experience Cloud computing technologies (Azure/AWS). Knowledge Windows Server 2008 & 2012 C# added advantage. Good understanding building scalable system. Practical, hands-on experience data platforms tool. Strong critical thinking, problem-solving, programming skills computer science knowledge. Interested applicants please send updated resume feroz@shellinfotech.com Please mention you’re present expected salary, availability."
2,"Senior Software Engineers deliver high-quality functional applications websites satisfy business needs requirements clients. You actively participate software development lifecycle, debug applications, configure existing systems. We like people love code history building programs applications. You actively research creative solutions problems. Responsibilities • Develop deploy fully functional applications based business technical specifications • Analyze overall development work plan tasks maximum efficiency reducing risk • Write clean, testable code including unit tests. • Debug applications websites • Oversee team’s technical deliverables, reviewing code coaching improve skills • Work multiple client partner teams implement cohesive end-to-end experiences • Document development operational procedures Requirements • Degree Computer Science, Engineering equivalent • 5+ years professional software development experience • Demonstrable ability build end-to-end solution using enterprise technologies (E.g. .NET, Java, Salesforce, etc.) • Experience creating enterprise-level applications • Familiarity agile environments • Excellent troubleshooting problem-solving skills • Good communication skills Kindly indicate last drawn salary notice period."
3,"Please note: Due security nature client, interested applicants must Singaporean PR. About Our Client They specialist Cyber Security Product company R&D division formed talented group cyber security experts, responsible delivering cutting-edge projects using innovative active defense methods. About Security Engineer Working alongside research team, responsible developing PoC transforming research security products. It expectation continuously develop knowledge specific field cyber security order establish domain expert Duties include Software Development, technical design reviews, integration, testing documentation, addition acting technical advisor research department Experience Required: Proficient high level programming / scripting language (Java, .Net, C++, python, bash) Full stack developer Database Presentation layer. Complete SDLC. Experience Research Product Development Environment A keen knowledge interest Cyber Security, understanding threat landscape (e.g. MitM, Ransomware), attack vectors active defence Experienced Software Security Reviews Vulnerability Assessment Ability multi-task, engaged various projects simultaneously Relevant Security Certifications would desirable Und

In [179]:
stop_words_l=stopwords.words('english')
documents_df['documents_cleaned']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split()))

In [175]:
documents_df=pd.DataFrame(documents_df,columns=['documents_cleaned'])

In [162]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

### Corpus Dataframe

In [180]:
documents_df['documents_cleaned']

0      requirements at least   months experience   open consider applicants experience technical skills  any programming language  c  c    java   net  python  good analytical  problem skills business level proficiency english liaise end users l  support                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

#### Tf-idf vectors

In [181]:
tfidfvectoriser=TfidfVectorizer(max_features=64)
tfidfvectoriser.fit(documents_df.documents_cleaned)
tfidf_vectors=tfidfvectoriser.transform(documents_df.documents_cleaned)

In [182]:
tfidf_vectors.shape

(118, 64)

In [183]:
tfidf_vectors=tfidf_vectors.toarray()
print (tfidf_vectors[0])

[0.         0.         0.         0.26874799 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.36638399 0.         0.         0.
 0.         0.2962928  0.27171355 0.         0.23044369 0.
 0.         0.         0.33033348 0.         0.         0.
 0.         0.         0.23498792 0.         0.         0.
 0.21560272 0.         0.         0.         0.         0.
 0.44778186 0.         0.         0.         0.         0.29442962
 0.         0.         0.         0.28424261 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


##### Every document has been converted into a 64 dimensional vector. As we set the max_features=64

### Pairwise similarity

##### Pairwise cosine similarity would just be the dot product of the vectors becasue tf-idf vectors from sklearn are already normalized and L2 norm of these vectors is 1. So denominator of cosine similiarity formula is 1 in this case.

In [184]:
pairwise_similarities=np.dot(tfidf_vectors,tfidf_vectors.T)
pairwise_differences=euclidean_distances(tfidf_vectors)

In [185]:
print (tfidf_vectors[0])
print (pairwise_similarities.shape)
print (pairwise_similarities[0][:])

[0.         0.         0.         0.26874799 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.36638399 0.         0.         0.
 0.         0.2962928  0.27171355 0.         0.23044369 0.
 0.         0.         0.33033348 0.         0.         0.
 0.         0.         0.23498792 0.         0.         0.
 0.21560272 0.         0.         0.         0.         0.
 0.44778186 0.         0.         0.         0.         0.29442962
 0.         0.         0.         0.28424261 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
(118, 118)
[1.         0.1507925  0.51638553 0.11075066 0.84839137 0.30674492
 0.29193753 0.11840473 0.17906158 0.11075066 0.11075066 0.34910695
 0.26710783 0.19913315 0.43710286 0.12393251 0.14838997 0.19344567
 0.23792929 0.32341131 0.09421267 0.17373079 0.12008003 0.16727807
 0.28889262 0.16505627 0.17671157 0.26710783 0.52245714 0.22670202
 0.1972059

##### similarity is highest, 1 at index 0 becasue they are the same documents

In [186]:
def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Document: {documents_df.iloc[doc_id]["documents"]}')
    print ('\n')
    print (f'Similar Documents using {matrix}:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print('\n')
        print (f'Document: {documents_df.iloc[ix]["documents"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

In [187]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Cosine Similarity:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Cosine Similarity : 0.848391369282737


Document: • Basic programming knowledge experience (gained commercially academically) • Good knowledge C/C++/Java/.NET/SQL. • Candidates interested build career programming apply necessary training provided. • Good commun

In [170]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Euclidean Distance:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Euclidean Distance : 0.5506516697827457


Document: • Basic programming knowledge experience (gained commercially academically) • Good knowledge C/C++/Java/.NET/SQL. • Candidates interested build career programming apply necessary training provided. • Good com

#### word2vec embeddings

In [139]:
from keras.preprocessing.text import Tokenizer
import gensim
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\icyro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [140]:
# tokenize and pad every document to make them of the same size
tokenizer=Tokenizer()
tokenizer.fit_on_texts(documents_df.documents_cleaned)
tokenized_documents=tokenizer.texts_to_sequences(documents_df.documents_cleaned)
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
vocab_size=len(tokenizer.word_index)+1

print (tokenized_paded_documents[0])

[   6   96 1167    1  328  906  123    1   22    8   15  175    7    7
    9   49  100   30  255  112    8   26  107  237  238  613   61  271
  614   33    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [143]:
document_word_embeddings.shape

(118, 64, 300)

In [144]:
# tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents

document_embeddings=np.zeros((len(tokenized_paded_documents),300))
words=tfidfvectoriser.get_feature_names()

for i in range(len(document_word_embeddings)):
    for j in range(len(words)):
        document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
        
document_embeddings=document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)
    

c:\Users\icyro\anaconda3\envs\cuda_3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


ValueError: operands could not be broadcast together with shapes (300,) (100,) (300,) 

In [145]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [146]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Cosine Similarity:


Document: • Proficient WPF, C# .Net, OO design. • Design simple, efficient intuitive UI • Attention detail • Able debug complex event driven application • Understanding computing science fundamentals – concurrency, algorithms, operating systems. • Mathematical aptitude ambition learn mathematics financial derivatives • Process tools produce well written low defect rate code. Collaborative development • At least 5 years depth commercial C# rich development experience. • At least 3 years experience UI design implementation using WPF • Experience developing supporting mission critical applications. • Knowledge FX advantage appreciation product structure workflow. • Experience working mature

In [147]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Euclidean Distance:


Document: 1. Pre-requisite Analyst Programmer 1.1 Required Competencies a. Experience four following skills: Java, EJB, JSP, JSF, Hibernate, JEE 5 above, Struts, PL/SQL, Oracle database version 11g 12c, XML b. Experience handling system-to-system backend interfaces highly preferred c. Experience software database design, Weblogic application server, UML modelling tools preferred d. Proficient spoken written English e. Good interpersonal communication skills. 1.2 Required Working Experience a. Minimum 3 years experience four following skills: Java, EJB, JSP, JSF, Hibernate, JEE 5 above, Struts, PL/SQL, Oracle database version 11g 12c, XML b. Knowledge Experience Procurement processes fin

#### Glove embeddings

In [148]:
# reading Glove word embeddings into a dictionary with "word" as key and values as word vectors

embeddings_index = dict()

with open('glove.6B.100d.txt',encoding='utf8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    

In [149]:
# creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 

embedding_matrix=np.zeros((vocab_size,100))

for word,i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [150]:
# tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents

document_embeddings=np.zeros((len(tokenized_paded_documents),100))
words=tfidfvectoriser.get_feature_names()

for i in range(documents_df.shape[0]):
    for j in range(len(words)):
        document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
        
document_embeddings=document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)

c:\Users\icyro\anaconda3\envs\cuda_3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [151]:
document_embeddings.shape

(118, 100)

In [152]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [153]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Cosine Similarity:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Cosine Similarity : 0.9774655643764029


Document: Job Description: The objective position develop improved version current video analytics platform (VAP) used upcoming strategic projects. Job Responsibilities: · Responsible development, implementation maintena

In [154]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Euclidean Distance:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Euclidean Distance : 0.8285753146738027


Document: Job Description: The objective position develop improved version current video analytics platform (VAP) used upcoming strategic projects. Job Responsibilities: · Responsible development, implementation mainte

#### Doc2vec model

In [155]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [156]:
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(documents_df.documents_cleaned)]

In [157]:
model_d2v = Doc2Vec(vector_size=100,alpha=0.025, min_count=1)
  
model_d2v.build_vocab(tagged_data)

for epoch in range(100):
    model_d2v.train(tagged_data,
                total_examples=model_d2v.corpus_count,
                epochs=model_d2v.epochs)

In [158]:
document_embeddings=np.zeros((documents_df.shape[0],100))

for i in range(len(document_embeddings)):
    document_embeddings[i]=model_d2v.docvecs[i]

C:\Users\icyro\AppData\Local\Temp\ipykernel_24600\1686857689.py:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  document_embeddings[i]=model_d2v.docvecs[i]


In [159]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [160]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Cosine Similarity:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Cosine Similarity : 0.8555321446882361


Document: Please note: Interested Applicants must Singaporean PR. About Our Client Our client specialist Cyber Security Product company R&D division formed talented group cyber security experts, responsible delivering cu

In [161]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Euclidean Distance:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Euclidean Distance : 9.609303319141995


Document: • Basic programming knowledge experience (gained commercially academically) • Good knowledge C/C++/Java/.NET/SQL. • Candidates interested build career programming apply necessary training provided. • Good comm

#### BERT model

In [162]:
from sentence_transformers import SentenceTransformer

In [163]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [164]:
document_embeddings = sbert_model.encode(documents_df['documents_cleaned'])

In [165]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [166]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Cosine Similarity:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Cosine Similarity : 0.8928866386413574


Document: At least 2 years working experience IT, 1 years Java developer complex web & web mobile applications. Must experience Java, Angular 4 , Restful Web Services, Spring , Hibernate, Oracle DB Strong software develo

In [167]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Euclidean Distance:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Euclidean Distance : 7.263776779174805


Document: At least 2 years working experience IT, 1 years Java developer complex web & web mobile applications. Must experience Java, Angular 4 , Restful Web Services, Spring , Hibernate, Oracle DB Strong software devel

In [47]:
documents_df['documents_cleaned'][0]

KeyError: 'documents_cleaned'

In [24]:
import tensorflow as tf
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

embeddings = embed(
    documents_df['documents_cleaned']
    )
print(embeddings)

tf.Tensor(
[[-0.00128172 -0.06747271 -0.04020526 ... -0.05906114 -0.0410772
  -0.05739686]
 [-0.04347231 -0.0528002  -0.03359879 ...  0.05257256 -0.05268412
  -0.04811721]
 [-0.00258304 -0.05699786  0.0406852  ... -0.04435417 -0.05697822
  -0.01906533]
 ...
 [ 0.02980568 -0.05163132  0.05122544 ...  0.05160279 -0.05164138
  -0.03851214]
 [-0.05205894 -0.05314077  0.03159722 ...  0.00172458 -0.05765938
  -0.04834338]
 [ 0.04473758 -0.06130524  0.05796009 ... -0.02593927 -0.06065044
   0.00201276]], shape=(118, 512), dtype=float32)


In [1]:
from scipy.spatial import distance
for i in range (len(documents_df)):
    for j in range(len(documents_df)):
        print(i,distance.cosine(embeddings[i],embeddings[j]))
    #print(1-distance.cosine(embeddings[0],embeddings[i]))
"""print(1 - distance.cosine(embeddings[0], embeddings[1]))
print(1 - distance.cosine(embeddings[0], embeddings[2]))
print(1 - distance.cosine(embeddings[1], embeddings[2]))"""

NameError: name 'documents_df' is not defined

In [31]:
from scipy.spatial import distance
for i in range (len(documents_df)):
    print(1-distance.cosine(embeddings[0],embeddings[i]))
"""print(1 - distance.cosine(embeddings[0], embeddings[1]))
print(1 - distance.cosine(embeddings[0], embeddings[2]))
print(1 - distance.cosine(embeddings[1], embeddings[2]))"""

1
0.5009726881980896
0.5228816270828247
0.4483901262283325
0.8396403193473816
0.3532962501049042
0.49935346841812134
0.4764935374259949
0.4878048300743103
0.4411826729774475
0.4360313415527344
0.4945048987865448
0.4972308576107025
0.3894360065460205
0.36636656522750854
0.4301057755947113
0.502716600894928
0.4816852807998657
0.39073970913887024
0.5325890779495239
0.5659420490264893
0.3944198787212372
0.4132324159145355
0.34199073910713196
0.4098392128944397
0.36798641085624695
0.431640088558197
0.4972308874130249
0.6855246424674988
0.4629843235015869
0.4043830633163452
0.369164377450943
0.60545814037323
0.4887065291404724
0.41016966104507446
0.24935109913349152
0.25371870398521423
0.4097170829772949
0.5236639380455017
0.5350213646888733
0.47970980405807495
0.4421049654483795
0.5875534415245056
0.5466405153274536
0.5290811061859131
0.45449134707450867
0.3485354781150818
0.4972308576107025
0.5219470858573914
0.3548574149608612
0.5600389242172241
0.3722469210624695
0.5743122100830078
0.434

'print(1 - distance.cosine(embeddings[0], embeddings[1]))\nprint(1 - distance.cosine(embeddings[0], embeddings[2]))\nprint(1 - distance.cosine(embeddings[1], embeddings[2]))'